<a href="https://colab.research.google.com/github/peppievt/Gated-training/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from matplotlib import style

import librosa
import librosa.display
import IPython.display as ipd
import keras

from numpy import dot
from numpy.linalg import norm
from google.colab import drive

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import LSTM, Embedding
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization

from keras.utils import np_utils, to_categorical
import pickle
from keras.layers import TimeDistributed

from sklearn.preprocessing import MinMaxScaler
import scipy.sparse as sparse
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf


In [ ]:
#adres = r'C:\Users\pepij\Documents\Uni\Datascience\Thesis\baldey_audio\real_compound'
#in colab:
adres = r'/content/gdrive/My Drive/Colab Notebooks/data/'
learning_Rate = 0.002
one_hot = True
style.use('dark_background')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#lengte = output dimensie
lengte = 2260
ones = 1
df = None
if df is not None:
    del df

In [ ]:
def create_Y():
    woordenlijst = []
    for i in dir_list:
        if i.endswith('.wav'):
            woordenlijst.append(i[:-4])
    return woordenlijst

def create_RandomVector():
    Vec = np.zeros(lengte)
    randomlist = random.sample(range(0, lengte), ones)
    for i in randomlist:
        Vec[i] = 1
    return np.asarray(Vec)

def create_map_Vectors(woordenlijst):
    omzet_df = pd.DataFrame(columns = ['woord', 'vector'])
    woorden, vectoren, check_doubles = [],[],[]
    c = 0
    for i in set(woordenlijst):
        woorden.append(i)
        vecie = create_RandomVector()
        while any((vecie == x).all() for x in vectoren):
            vecie = create_RandomVector()
        else:
            vectoren.append(vecie)
        c+=1
    print("unique vectors: ",len(np.unique(vectoren, axis=0)))
    omzet_df['woord'] = woorden
    omzet_df['vector'] = vectoren
    dictionary = dict(zip(omzet_df['woord'], omzet_df['vector']))
    return dictionary

def solve_table(df, di):
    df['Y'] = df['name'].map(di)
    return df

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# for trainingset:
df_features = pd.read_pickle(adres + 'Baldey_features/real_2_specto_baldey.pkl')
df_names = pd.read_pickle(adres + 'Baldey_features/real_2_specto_baldey_names.pkl')

df = pd.DataFrame(df_features['feature'].values.tolist(), columns=['feature'])
df['name'] = df_names
dictionary = create_map_Vectors(df['name']) 

df = solve_table(df, dictionary)

df['Y']=df['Y']/ones

print("trainingset generated")

#for validation sets:
df_2 = pd.DataFrame(columns=['valX', 'val_name'])
val_X = pd.read_pickle(adres + 'Baldey_features/fake_2_specto_baldey.pkl')
val_name = pd.read_pickle(adres + 'Baldey_features/fake_2_specto_baldey_names.pkl')
df_2 = pd.DataFrame(val_X['feature'].values.tolist(), columns=['valX'])
df_2['val_name'] = val_name
df_2['valY'] = df_2['val_name'].map(dictionary)
df_2['valY']= df_2['valY']/ones

print("training: ",len(df))
print("validation: ", len(df_2))

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


unique vectors:  2260
trainingset generated
training:  2260
validation:  4520


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [ ]:
#Create train/validation split:

X_train = df['feature']
y_train = df['Y']
lennie = len(df_2['valX'])
X_val = df_2['valX'][0:int(0.5*lennie)]
y_val = df_2['valY'][0:int(0.5*lennie)]
X_test = df_2['valX']
y_test = df_2['valY']

print("yval = ", y_val.shape)
print("ytest = ",y_test.shape)

yval =  (2260,)
ytest =  (4520,)


In [ ]:
''' scaler = MinMaxScaler(feature_range=(-1, 1))
for i in range(len(X_train)):
    X_train[i] = np.array(scaler.fit_transform(X_train[i]))
for i in range(len(X_val)):
    X_val[i] = np.array(scaler.fit_transform(X_val[i]))
for i in range(len(X_test)):
    X_test[i] = np.array(scaler.fit_transform(X_test[i]))
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
print("validation size: ", len(X_val))
print("test size: ", len(X_test))'''

' scaler = MinMaxScaler(feature_range=(-1, 1))\nfor i in range(len(X_train)):\n    X_train[i] = np.array(scaler.fit_transform(X_train[i]))\nfor i in range(len(X_val)):\n    X_val[i] = np.array(scaler.fit_transform(X_val[i]))\nfor i in range(len(X_test)):\n    X_test[i] = np.array(scaler.fit_transform(X_test[i]))\ny_train = np.array(y_train)\ny_val = np.array(y_val)\ny_test = np.array(y_test)\nprint("validation size: ", len(X_val))\nprint("test size: ", len(X_test))'

In [ ]:
#AANGEPAST VOOR SPINOFF-EXPERIMENT:
# choose 200 options
options = []
for i in range(len(y_train)):
    if i < 201:
        options.append(np.argmax(y_train[i]))
# generate same options for val and test
argmax_opties = []
for i in range(len(y_val)):
       if np.argmax(y_val[i]) in options:
           argmax_opties.append(i)
print(len(argmax_opties))
print(argmax_opties[0:50])
# put correct samples in new sets:
print("-------------------------")
print("training size: ", len(X_train))
print("val size: ", len(X_val))
print("test size: ", len(X_test))

X_train = X_train[0:200]
y_train = y_train[0:200]

X_val = [X_val[i] for i in range(len(X_val)) if i in argmax_opties]
y_val = [y_val[i] for i in range(len(y_val)) if i in argmax_opties]

X_val = X_val[0:200]
y_val = y_val[0:200]
print("-------------------------")
print("training size: ", len(X_train))
print("val size: ", len(X_val))
print("test size: ", len(X_test))

382
[1, 2, 4, 10, 11, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
-------------------------
training size:  2260
val size:  2260
test size:  4520
-------------------------
training size:  200
val size:  200
test size:  4520


In [ ]:
print(X_train[0].shape)
print(np.argmax(y_train[0]))
print(np.max(y_train[0]))

(128, 68)
823
1.0


In [ ]:
# unpack the x = [nr_of_samples,] array to: [nr_of_samples, length]
def unpack_arrays(x):
    data_x = []
    for i in range(len(x)):
        temp = [x[i]]
        data_x.extend(temp)
    return np.asarray(data_x)
    
#computes the cosine similarity between two vectors:    
def cosine_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

#given a sample and the names/random vectors dictionary: calculate the cosine similarity for the entire list.
def cosine_sim_list(a, b):
    lijst = [cosine_sim(a,x) for x in b]
    return lijst

#full samples added: if gates = False, else; split in 1/nr_of_gates
def create_gated_samples(X_train, y_train, nr_of_gates=2, gates = True):
    ey = []
    ex = pd.Series()
    counter = 0
    for x in X_train:
        step_number = int(len(x[0]))
        if gates == False:
            step = step_number
        else:
            step = int(step_number/nr_of_gates)
        x = x.T
        for i in range(1,nr_of_gates,1):
            ex= pd.Series([x[:step*i][:].T])
            X_train[len(X_train)] = ex[0]
            #ey aanpassen met Entropywaarde 5 -> [0/5, 0.5/sum(samples) voor alle andere samples)]
            y = y_train[counter]*0.5
            y = [0.5/len(y) if i< 0.4 else i for i in y]
            ey.append(y)
        counter +=1
    y_train = unpack_arrays(y_train)
    y_train = np.concatenate([y_train, ey])
    return X_train, y_train

In [ ]:
#Create augmented samples:
print("length of X and Y before: ", len(X_train), len(y_train))
#X_train, y_train = create_gated_samples(X_train, y_train, gates = False)
print("length of X and Y after: ", len(X_train), len(y_train))
#print(y_train[3])

length of X and Y before:  200 200
length of X and Y after:  200 200


In [ ]:
#check:
print(X_train[0].shape)
print(X_train[len(X_train)/2].shape)

#perform transpose operations:

print("shape before: ", X_train[0].shape)
print(X_train[len(X_train)/2].shape)

X_train = [x.T for x in X_train]
X_val = [x.T for x in X_val]
X_train = pd.Series(X_train)
X_val = pd.Series(X_val)
print("shape after: ", X_train[0].shape)
print(X_train[len(X_train)/2].shape)

(128, 68)
(128, 54)
shape before:  (128, 68)
(128, 54)
shape after:  (68, 128)
(54, 128)


In [ ]:
print(len(X_train[0][0]))

128


In [ ]:
#if one_hot is False:
#    print("false")
#    y_train = unpack_arrays(y_train)


In [ ]:
def model_LSTM():
    model = Sequential()
    model.add(LSTM(512, input_shape=(None,128)))
    model.add(Dense(len(y_train[0])))
    model.add(Activation('softmax'))
    model.summary()
    return model

In [ ]:
# functions for plotting the results.
def plot_model(model_history):
    if ones == 1:
        metric = 'accuracy'
    else:
        metric = 'top_k_categorical_accuracy'
    plt.plot(model_history.history['loss'], color='blue')
    plt.plot(model_history.history['val_loss'], color ='red')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.plot(model_history.history[metric], color='blue')
    plt.plot(model_history.history['val_'+ metric], color = 'red')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.show()

def save_other_info():
    np.save("X_train", X_train)
    np.save("X_val", X_val)
    np.save("X_test", X_test)
    np.save("y_train", y_train)
    np.save("y_val", y_val)
    np.save("y_test", y_test)
    with open('dict.p', 'wb') as fp:
        pickle.dump(dictionary, fp, protocol=pickle.HIGHEST_PROTOCOL)


# Save model and weights
def save_model(model_name, model):
    print(model_name)
    model_name = model_name
    %cd /content/gdrive/My Drive/Colab Notebooks/data
    save_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/data', 'saved_models')
    print(save_dir)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    np.save(model_path + "/X_train", X_train)
    np.save(model_path + "/X_val", X_val)
    np.save(model_path + "/X_test", X_test)
    np.save(model_path + "/y_train", y_train)
    np.save(model_path + "/y_val", y_val)
    np.save(model_path + "/y_test", y_test)
    
    with open(model_path + '/dict.p', 'wb') as fp:
        pickle.dump(dictionary, fp, protocol=pickle.HIGHEST_PROTOCOL)
    print('Save model and weights at %s ' % model_path)

In [ ]:
#batch generator class:
from random import random
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras models'
    def __init__(self, X_train, labels, batch_size=1, dim=(len(X_train),None,len(X_train[0][0])), shuffle=False, gated=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.X_train = X_train
        self.on_epoch_end()
        self.shuffle=shuffle
        self.gated = gated

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X_train) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        ki = self.indexes[index]
        X = np.expand_dims(X_train[ki], axis=0)
        yi = y_train[ki]
        #alleen aanzetten voor minitest:
        if self.gated:
            rand = random()
            if rand < 0.5 and rand > 0.05:
                length = rand*2
                X = X[:,0:int(length*len(X[0])) ,:]
                yi = yi*length
                yi = [length/len(yi) if i< length else i for i in yi]
        y = np.expand_dims(yi, axis=0)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(X_train))

In [ ]:
training_generator = DataGenerator(X_train, y_train, gated=True)
validation_generator = DataGenerator(X_val, y_val)
print("lengte trainingset: ",len(training_generator))
print("lengte validatieset: ",len(validation_generator))

lengte trainingset:  200
lengte validatieset:  200


In [ ]:
def run(model, learning_Rate):
    if ones == 1:
        metric = 'accuracy'
    else:
        metric = accie
    #Noam decay of learning rate
    print(learning_Rate)
    global_Step = 0
    step = (global_Step + 1)
    warmup_Steps = 400.0
    learning_Rate *= warmup_Steps ** 0.5 * np.minimum(step * warmup_Steps**-1.5, step**-0.5)
    #Adam optimizer
    opt = keras.optimizers.Adam(lr=learning_Rate)
    accie = tf.keras.metrics.TopKCategoricalAccuracy(k=ones, name="top_k_categorical_accuracy", dtype=None)
    #compile and build model:
    model.compile(loss='cosine_similarity', optimizer=opt, metrics=metric)
    model_history = model.fit(training_generator, steps_per_epoch=len(X_train), validation_data=validation_generator, epochs=600,  verbose=0)
    plot_model(model_history) 
    print(learning_Rate)
    return model, learning_Rate

In [ ]:
model = model_LSTM()
learning_Rate = 0.002
x, learning_Rate = run(model, learning_Rate)
save_model("(2500,1)_mini_test_3-3_gated_600", x)
epochs = 600

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 512)               1312768   
_________________________________________________________________
dense_21 (Dense)             (None, 2260)              1159380   
_________________________________________________________________
activation_21 (Activation)   (None, 2260)              0         
Total params: 2,472,148
Trainable params: 2,472,148
Non-trainable params: 0
_________________________________________________________________
0.002


In [ ]:
hist = model.history.history['accuracy']
hist_loss = model.history.history['loss']
print("accuracy: ",hist[-1])
print("loss: ", hist_loss[-1])
np.save("model_acc_" + str(epochs), hist)
np.save("model_loss_"+ str(epochs), hist_loss)
np.save("model_val_acc_" + str(epochs), model.history.history["val_accuracy"])
np.save("model_val_loss_"+ str(epochs), model.history.history["val_loss"])

In [ ]:
print(model.history.history['loss'])

[-0.029748626053333282, -0.03132721409201622, -0.031633708626031876, -0.03196489438414574, -0.03241903707385063, -0.03309467062354088, -0.03409312665462494, -0.0354282408952713, -0.037080615758895874, -0.03899482637643814, -0.04114321246743202, -0.043620314449071884, -0.046325474977493286, -0.049249403178691864, -0.052373871207237244, -0.05574239417910576, -0.05928408354520798, -0.06302028894424438, -0.06699711084365845, -0.07086554914712906, -0.0755590870976448, -0.07997600734233856, -0.08483091741800308, -0.0900847539305687, -0.09551087021827698, -0.10071557760238647, -0.10630272328853607, -0.11247047781944275, -0.11843384802341461, -0.12430411577224731, -0.13151104748249054, -0.13789421319961548, -0.14396128058433533, -0.15143325924873352, -0.15878303349018097, -0.1658685803413391, -0.1728535294532776, -0.18083050847053528, -0.18848837912082672, -0.19652985036373138, -0.2051280438899994, -0.21174652874469757, -0.22116711735725403, -0.22907541692256927, -0.23855939507484436, -0.24655

In [ ]:
#oefen sample:
sample = np.expand_dims(X_train[0], axis=0)
preds = model.predict(sample, verbose=0)
print("similarity with target: ", cosine_sim(preds, y_train[0]))


similarity with target:  [0.09675417]
